# Householder and Givens Tools

The plan is to implement a number of things in pure Julia to provide access to the AD tools.

1. Single step in place two-sided Householder reduction
1. XXX

## Householder

## Householder Start

In [87]:
using LinearAlgebra
m=5;
A = rand(m,m); H=copy(A)
v=zeros(m)
# 1st column
i=1
v=zeros(m)
v[i+1:m] = H[i+1:m,i]
v[i+1]-=norm(v)
v /= norm(v)
H -= 2.0*(H*v)*v'
H -= 2.0*v*(H'*v)'
# 2nd column
i=2
v=zeros(m)
v[i+1:m] = H[i+1:m,i]
v[i+1]-=norm(v)
v /= norm(v)
H -= 2.0*(H*v)*v'
H -= 2.0*v*(H'*v)'
# 3rd column 
# with mutation and fixed storage for v as an m vector
i=3
#v=0*v
v[i+1:m] = H[i+1:m,i]
v[i+1]-=norm(v[i+1:m])
v[i+1:m] /= norm(v[i+1:m])
H[i+1:m,i:m] -= 2.0*v[i+1:m]*(H[i+1:m,i:m]'*v[i+1:m])'
H[1:m,i+1:m] -= 2.0*(H[1:m,i+1:m]*v[i+1:m])*v[i+1:m]'
# Check
print(norm(eigen(A).values-eigen(H).values))
H

3.589387267483526e-15

5×5 Matrix{Float64}:
  0.0134093     1.11518      0.194692      0.120225  -0.461721
  1.08501       1.81417      0.88617      -0.505577   0.0925293
  1.54373e-17   1.32182      0.698382      0.747266  -0.00751425
  7.72658e-17  -1.66533e-16  0.12028      -0.216856   0.224815
 -7.82153e-17  -3.88578e-16  1.38778e-17   0.191219   0.293834

## Loop Script

In [88]:
using LinearAlgebra
m=5;
A = rand(m,m); H=copy(A)
v=zeros(m)
for i in 1:m-2
    v[i+1:m] = H[i+1:m,i]
    v[i+1]-=norm(v[i+1:m])
    v[i+1:m] /= norm(v[i+1:m])
    H[i+1:m,i:m] -= 2.0*v[i+1:m]*(H[i+1:m,i:m]'*v[i+1:m])'
    H[1:m,i+1:m] -= 2.0*(H[1:m,i+1:m]*v[i+1:m])*v[i+1:m]'
end
# Check
print(norm(eigen(A).values-eigen(H).values))
H

2.16724662841394e-15

5×5 Matrix{Float64}:
  0.826014     0.814989     0.501048      0.345439    0.0581925
  1.37216      1.40698      0.425821      1.11297    -0.158519
 -5.55112e-17  1.17193      0.31592       0.241514   -0.534538
  0.0          0.0          0.606833     -0.0662567  -0.509651
 -2.22045e-16  1.11022e-16  2.22045e-16  -0.416923    0.0215552

## In place function with workspace.

The fixed size workspace is called v.  

In [89]:
function Hess1(H,v,i,m)
    v[i+1:m] = H[i+1:m,i]
    v[i+1]-=norm(v[i+1:m])
    v[i+1:m] /= norm(v[i+1:m])
    H[i+1:m,i:m] -= 2.0*v[i+1:m]*(H[i+1:m,i:m]'*v[i+1:m])'
    H[1:m,i+1:m] -= 2.0*(H[1:m,i+1:m]*v[i+1:m])*v[i+1:m]'
end

Hess1 (generic function with 2 methods)

In [96]:
# import Pkg; Pkg.add("PlotlyJS")

   Resolving package versions...
   Installed AssetRegistry ───────── v0.1.0
   Installed JSExpr ──────────────── v0.5.4
   Installed URIParser ───────────── v0.4.1
   Installed Mustache ────────────── v1.0.17
   Installed PlotlyBase ──────────── v0.8.19
   Installed WebSockets ──────────── v1.5.9
   Installed Hiccup ──────────────── v0.2.2
   Installed Blink ───────────────── v0.12.5
   Installed PlotlyJS ────────────── v0.18.10
   Installed Mux ─────────────────── v0.7.6
   Installed Pidfile ─────────────── v1.3.0
   Installed FunctionalCollections ─ v0.5.0
   Installed Kaleido_jll ─────────── v0.2.1+0
   Installed Lazy ────────────────── v0.15.1
   Installed HTTP ────────────────── v0.9.17
   Installed BinDeps ─────────────── v1.0.2
   Installed Widgets ─────────────── v0.6.6
   Installed WebIO ───────────────── v0.8.21
    Updating `C:\Users\struther\.julia\environments\v1.6\Project.toml`
  [f0f68f2c] + PlotlyJS v0.18.10
    Updating `C:\Users\struther\.julia\environments\v1.6\Mani

In [99]:
import Pkg; Pkg.add("Conda")
using Conda
Conda.pip_interop(true)
Conda.pip("install", "webio_jupyter_extension")

   Resolving package versions...
    Updating `C:\Users\struther\.julia\environments\v1.6\Project.toml`
  [8f4d0f93] + Conda v1.9.1
  No Changes to `C:\Users\struther\.julia\environments\v1.6\Manifest.toml`
Precompiling project...
  ✓ PolynomialBases
  ✓ KernelDensity
  ✓ DSP
  ✓ ToeplitzMatrices
  ✓ Gadfly
  ✓ Gridap
  6 dependencies successfully precompiled in 21 seconds (303 already precompiled, 2 skipped during auto due to previous errors)
  2 dependencies precompiled but different versions are currently loaded. Restart julia to access the new versions
┌ Info: Downloading miniconda installer ...
└ @ Conda C:\Users\struther\.julia\packages\Conda\2lg2O\src\Conda.jl:210
┌ Info: Installing miniconda ...
└ @ Conda C:\Users\struther\.julia\packages\Conda\2lg2O\src\Conda.jl:221


LoadError: failed process: Process(`[4m'C:\Users\struther\AppData\Local\Temp\installer.exe'[24m [4m/S[24m [4m--no-shortcuts[24m [4m/NoRegistry=1[24m [4m/AddToPath=0[24m [4m/RegisterPython=0[24m [4m'/D=C:\Users\struther\.julia\conda\3'[24m`, ProcessExited(2)) [2]


In [100]:
using LinearAlgebra, PlotlyJS
m=7;
A = rand(m,m); H=copy(A); v=zeros(m);
for i in 1:m-2
    Hess1(H,v,i,m)
end
print(norm(eigen(H).values -eigen(A).values))
plot_matrix(H)

6.2920905091964875e-15

LoadError: UndefVarError: plot_matrix not defined

In [38]:
(i,j)=(2,3)
A[i,j]

1.142665811032841